In [1]:
from glob import glob

import dask.bag as db

In [2]:
b = db.from_sequence([1, 2, 3, 4, 5, 6])
b

dask.bag<from_se..., npartitions=6>

In [3]:
def squarefx(x):
    y = x*x
    return y

In [4]:
bb = b.map(squarefx)
bb

dask.bag<map-squ..., npartitions=6>

In [5]:
bb.compute()

[1, 4, 9, 16, 25, 36]

In [6]:
def loadtxt(txtfile):
    t = open(txtfile).read().splitlines()
    t = list(map(int, t))
    return t

In [7]:
#try dask on loading text files
b=db.from_sequence(['t1.txt','t2.txt','t3.txt','t4.txt']).map(loadtxt)
b

dask.bag<map-loa..., npartitions=4>

In [8]:
b.compute()

[[1, 1], [2, 2], [3, 3], [4, 4]]

In [9]:
def squarefxlist(x):
    return [i ** 2 for i in x]

In [10]:
bb=b.map(squarefxlist)
bb.compute()

[[1, 1], [4, 4], [9, 9], [16, 16]]

In [11]:
def read_sl(fname):
    from nibabel.streamlines import load as load_trk
    import dipy.tracking.utils as dtu
    import numpy as np
    """
    Reads streamlins from file.
    """
    tgram = load_trk(fname)
    sl = list(dtu.move_streamlines(tgram.streamlines,
                                   np.eye(4), tgram.affine))
    return sl

In [12]:
ls /Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/

bundles_af.left.trk      bundles_cg.left.trk      bundles_mdlf.right.trk
bundles_af.right.trk     bundles_cg.right.trk     bundles_slf1.left.trk
bundles_cc_1.trk         bundles_cst.left.trk     bundles_slf1.right.trk
bundles_cc_2.trk         bundles_cst.right.trk    bundles_slf2.left.trk
bundles_cc_3.trk         bundles_ifof.left.trk    bundles_slf2.right.trk
bundles_cc_4.trk         bundles_ifof.right.trk   bundles_slf_3.left.trk
bundles_cc_5.trk         bundles_ilf.left.trk     bundles_slf_3.right.trk
bundles_cc_6.trk         bundles_ilf.right.trk    bundles_uf.left.trk
bundles_cc_7.trk         bundles_mdlf.left.trk    bundles_uf.right.trk


In [13]:
bundle_files = glob('/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/*.trk')

In [14]:
#do without dask
streamlines = read_sl(bundle_files[0]) 
streamlines[0].shape

(217, 3)

In [15]:
bundle_files

['/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_6.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cst.left.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_7.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_5.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf2.right.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.left.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.right.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_ifof.left.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_4.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_1.trk',
 '/Users/jpowe

In [16]:
#do with dask
streamlines = db.from_sequence([bundle_files[0]]).map(read_sl)
streamlines

dask.bag<map-rea..., npartitions=1>

In [17]:
#this takes an eternity, why?
streamlines.compute()

Process ForkPoolWorker-16:
Process ForkPoolWorker-17:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda2/envs/bundlenet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda2/envs/bundlenet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda2/envs/bundlenet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda2/envs/bundlenet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda2/envs/bundlenet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda2/envs/bundlenet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/a

KeyboardInterrupt: 